In [1]:
import sys
sys.path.insert(0, '../')

from minisom import MiniSom

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline

%load_ext autoreload

data = np.genfromtxt('iris.csv', delimiter=',', usecols=(0, 1, 2, 3))
data = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 1, data)
labels = np.genfromtxt('iris.csv', delimiter=',', usecols=(4), dtype=str)

# Initialization and training
som = MiniSom(5, 5, 4, sigma=3, learning_rate=0.5, 
              neighborhood_function='triangle', random_seed=10)

In [2]:
class_assignments = som.labels_map(data, labels)

def classify(som, data, class_assignments):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    winmap = class_assignments
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in data:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, labels)

som.pca_weights_init(X_train)
som.train_random(X_train, 5000, verbose=False)
class_assignments = som.labels_map(X_train, y_train)

print(classification_report(y_test, classify(som, X_test, class_assignments)))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       1.00      0.90      0.95        10
   virginica       0.94      1.00      0.97        15

    accuracy                           0.97        38
   macro avg       0.98      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38

